In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

loaders = CSVLoader(file_path= "Test-Sheet1.csv", encoding= "utf-8")
data = loaders.load()

In [2]:
data

[Document(page_content='question: What are the office hours of the bank?\nanswer: Summer Season (16th Magh - 15th Kartik)\nDetails:\n1. Sun-Thurs(Office Hours 9:30 A.M to 5:30 P.M and Transaction Hours 10:00 A.M to 4:00 P.M)\n2. Friday(Office Hours 9:30 A.M. to 3:00 P.M. and Transaction Hours 10:00 A.M. to 4:00 P.M)\n\nWinter Season (16th Kartik - 15th Magh)\nDetails:\n1. Sun-Thurs(Office Hours 9:30 A.M to 4:30 P.M and Transaction Hours 10:00 A.M to 3:00 P.M)\n2. Friday(Office Hours 9:30 A.M. to 3:00 P.M. and Transaction Hours10:00 A.M. to 2:00 P.M)', metadata={'source': 'Test-Sheet1.csv', 'row': 0}),
 Document(page_content='question: Company detail/company profile\nanswer: Muktinath Bikas Bank Limited (MNBBL) was established on 19th Paush 2063 B.S. (i.e. 3rd January 2007 A.D.), The Bank is licensed by the Central Bank of Nepal to operate as a “B” Class National Level financial institution with its Central Office at Kamaladi, Kathmandu, Nepal.Today, Muktinath Bikas Bank is known as the

In [5]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from langchain.vectorstores.milvus import Milvus
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap

load_dotenv()
host = os.getenv('HOST')
port = os.getenv('PORT')
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
generation_config = {
    "temperature": 0.9,
    "top_k": 2,
}

llm = ChatGoogleGenerativeAI(model = "gemini-pro", generation_config =generation_config)

output_parser = StrOutputParser()

loaders = CSVLoader(file_path= "/Volumes/Ajeet/LLM Projects/CSV-QnA/Test-Sheet1.csv", encoding= "utf-8")
data = loaders.load()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
database = Milvus(data, embeddings,connection_args={'host':host,'port':port},collection_name='CSV_READER')




prompt = """
### Instruction ###
You are a Virtual Assistant for Muktinath Bikas Bank. When a user asks a question you have to answer him based on the context and make sure
to provide all the details.
Context is given in triple back ticks: ```{context}```
Question is given in double back ticks:  ``{question}``
If the question is out of context, do not provide the wrong answer just say I am sorry i can't answer to that question.
"""

template = ChatPromptTemplate.from_template(prompt)


chain = RunnableMap(
    {
        "context": lambda x:database.similarity_search(x['question'], k = 8),
        "question": lambda x:x['question']
    }
)|template|llm|output_parser

def result(query:str):
    result = chain.invoke({'question':query})
    return {"text":result}



TypeError: Milvus.__init__() got multiple values for argument 'collection_name'

In [4]:

if __name__ == "__main__":
    query = input("Input your query")
    response= result(query= query)
    print("Response:",response)


Response: {'text': 'The board of directors of Muktinath Bikas Bank are: \n1. Bharat Raj Dhakal (Chairman)\n2. Narayan Kumar Shrestha (Director)\n3. Bharat Prasad Lamsal (Director)\n4. Shyam Sundar Sharma (Independent Director)\n5. Binod Kumar Sharma (Director)\n6. Saroja Shrestha Koirala (Director)'}
